## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = range(0, 1000)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
import pyspark

sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [5]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [6]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [7]:
# count
rdd.count()

1000

In [8]:
# first
rdd.first()

0

In [10]:
# take
rdd.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
# top
rdd.top(10)

[999, 998, 997, 996, 995, 994, 993, 992, 991, 990]

In [12]:
# collect
rdd.collect()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [13]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([6.88299862e-01, 3.92910741e-01, 2.21237012e+00, 2.39607123e-01,
       4.12354670e+00, 3.68673627e+00, 5.98969142e+00, 6.91206884e+00,
       2.16717824e+00, 4.07166855e+00, 8.55991459e+00, 1.00854351e+01,
       3.63114675e+00, 1.38682861e+01, 8.61539508e+00, 5.65988775e+00,
       1.68127704e+01, 8.14067524e+00, 1.35693794e+00, 1.84939803e+01,
       1.21446424e+00, 1.99046954e+01, 6.16035833e+00, 2.22504092e+01,
       6.61885483e+00, 3.31894051e+00, 2.33961820e+01, 2.36505316e+01,
       5.53262136e+00, 1.20019032e+00, 1.95569602e+01, 2.20561156e+00,
       1.62519444e+01, 1.49629204e+01, 1.86593890e+01, 8.56262437e+00,
       2.30486136e+01, 3.52974242e+01, 2.87031385e+01, 3.17621813e+01,
       5.68751286e+00, 1.38596063e+01, 1.58109465e+01, 6.73279283e+00,
       2.82829582e+01, 2.26207441e+01, 1.18309647e+00, 2.36899495e+01,
       3.15835123e+01, 4.19916697e+00, 1.27614952e+01, 1.85377621e+01,
       4.51922562e+01, 4.63591837e+01, 1.18278395e+01, 4.60525832e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [14]:
price_items = sc.parallelize(sales_figures, numSlices=10)


Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [15]:
def sales_tax(num):
    return num * 0.08

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [22]:
# perform action to retrieve rdd values
revenue_minus_tax.top(10)

[76.84571365159732,
 73.13425685641789,
 71.78081552386173,
 71.3973771097345,
 70.88213800560801,
 70.34265040625142,
 68.78606729329137,
 68.44948740419188,
 67.22863495868259,
 66.78285343490636]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [19]:
discounted = revenue_minus_tax.map(lambda x: .9 * x)

In [18]:
discounted.take(10)

[0.6194698759276028,
 0.3536196668682715,
 1.9911331068336746,
 0.21564641057423928,
 3.7111920288102147,
 3.3180626449677093,
 5.39072228127991,
 6.2208619553583615,
 1.950460411922676,
 3.664501692561182]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [23]:
discounted.map(lambda x: 1.1 * x).map(lambda x: 1.2 * x).top(15)

[91.29270781809763,
 86.88349714542446,
 85.27560884234774,
 84.8200840063646,
 84.20797995066232,
 83.5670686826267,
 81.71784794443015,
 81.31799103617998,
 79.86761833091491,
 79.33802988066876,
 78.95392429507233,
 78.49681769775594,
 78.32080179596534,
 78.0772028675911,
 78.06859143962811]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [24]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [25]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))

print(mapped.count())
print(mapped.take(10))

1000
[(0.688299862141781, 0.5699122858533947), (0.3929107409647461, 0.3253300935188098), (2.212370118704083, 1.8318424582869808), (0.23960712286026586, 0.19839469772830015), (4.123546698678016, 3.4142966665053973), (3.6867362721863435, 3.052617633370293), (5.989691423644344, 4.959464498777518), (6.912068839287068, 5.723192998929693), (2.16717823546964, 1.794423578968862), (4.071668547290202, 3.3713415571562875)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [27]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.688299862141781, 0.5699122858533947, 0.3929107409647461, 0.3253300935188098, 2.212370118704083, 1.8318424582869808, 0.23960712286026586, 0.19839469772830015, 4.123546698678016, 3.4142966665053973]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [30]:
# use the filter function
selected_items = price_items.filter(lambda x: x > 300)
selected_items.top(10)
# calculate total remaining in inventory 
selected_items.count()

350

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [31]:
selected_items.reduce(lambda x,y: x + y)


171157.80539777916

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [32]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(4, 321.9902218697895),
 (8, 304.2771820710812),
 (10, 336.93315628278293),
 (38, 313.8896119106414),
 (37, 319.7836655295107),
 (15, 365.0580403494256),
 (43, 341.26064998005785)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [37]:
# calculate how much each user spent
personal = sales_data.reduceByKey(lambda x, y: x + y)
personal.sortBy(lambda x: x[1],ascending = False).collect()


[(19, 7066.200673212205),
 (10, 6900.592011675384),
 (17, 6113.436481378805),
 (34, 6099.743533512764),
 (46, 5364.313702317658),
 (25, 5317.965516368449),
 (9, 4935.111995973161),
 (27, 4743.19205090762),
 (5, 4712.99358963104),
 (37, 4658.638081053802),
 (42, 4340.2242164648815),
 (3, 4316.935931328688),
 (24, 4292.141893683018),
 (40, 4291.877537344309),
 (33, 4260.90228793939),
 (1, 4143.814606930049),
 (29, 4008.011977066558),
 (47, 3960.4823912695374),
 (41, 3880.9797370858496),
 (45, 3805.9637360214247),
 (22, 3803.2139880894774),
 (2, 3561.865820348952),
 (11, 3559.59565750612),
 (13, 3537.561865504571),
 (30, 3530.753060432594),
 (18, 3412.2813705608846),
 (15, 3059.4572897487756),
 (7, 3053.3802543159827),
 (16, 2949.3874107011566),
 (39, 2914.279651019923),
 (12, 2843.6890578353523),
 (26, 2801.705255708194),
 (8, 2748.4650374333905),
 (35, 2679.209371986143),
 (32, 2574.524956812711),
 (38, 2505.1106865619186),
 (14, 2349.1808234574996),
 (50, 2255.5442728965963),
 (48, 200

In [ ]:
# sort the users from highest to lowest spenders


Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [38]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(9, 14),
 (21, 11),
 (33, 10),
 (46, 10),
 (31, 10),
 (16, 10),
 (23, 10),
 (20, 10),
 (25, 10),
 (47, 9),
 (15, 9),
 (34, 8),
 (18, 8),
 (49, 8),
 (39, 8),
 (19, 8),
 (7, 8),
 (41, 8),
 (6, 7),
 (44, 7),
 (12, 7),
 (22, 7),
 (42, 7),
 (10, 7),
 (40, 7),
 (38, 7),
 (14, 7),
 (11, 7),
 (4, 7),
 (3, 7),
 (26, 7),
 (35, 6),
 (2, 6),
 (28, 6),
 (1, 6),
 (8, 6),
 (13, 6),
 (30, 6),
 (43, 5),
 (36, 5),
 (24, 5),
 (27, 5),
 (45, 5),
 (5, 5),
 (37, 4),
 (32, 4),
 (48, 3),
 (17, 3),
 (50, 2),
 (29, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
